# Codebase for Data Preprocessing

## Things that have done
1. load raw data
2. data transformation (numeric and categorcial data)
3. univariate and multivariate exploratory data analysis (EDA)
4. correlation analysis
5. text preprocessing using NLP techniques
6. data export

In [ ]:
# load raw data
import pandas as pd
df=pd.read_csv("https://raw.githubusercontent.com/Alex-Mak-MCW/SpotifyDataScienceProject/main/Data/complete_data.csv")

In [ ]:
# print data
df.info()

In [ ]:
# check null and duplicate values

# identify null data
print("Number of null values in the dataset:\n{0}".format(df.isnull().sum())) 
# There are 26 missing value in lyrics--> will be handled

# identify duplciate data
print("\nNumber of duplicate data in the dataset:\n{0}".format(df.duplicated().sum())) # No duplicate values

In [ ]:
# check duration and duration_ms
print(df['duration'])
print(df['duration_ms'])

# drop duration_ms
df.drop(columns=['duration_ms'], inplace=True)
df.info()

In [ ]:
df.columns

In [ ]:
# drop object columns: track_href, analysis_url, Track ID, id, uri, and type
df.drop(columns=['track_href', 'analysis_url', 'track_ID', 'id', 'uri', 'type'], inplace=True)

print(df.shape) # (2457, 21)

In [ ]:
# drop preview url
# drop object columns: track_href, analysis_url, Track ID, id, uri, and type
df.drop(columns=['preview_url'], inplace=True)

print(df.shape) # (2457, 20)

In [ ]:
df.columns

In [ ]:
# DATA PREPPROCESSING (Part 1: numbers)

# 1. binary encode boolean feature (explicit) 
# print(df['explicit'].value_counts())
df['explicit']=df['explicit'].astype(int)
# print(df['explicit'].value_counts()) # 1719 F (0), 738 T (1)

In [ ]:
# # Descriptive/ Summary statistics for continuous data (15 columns)
# numeric_summary=df.describe().style.set_caption('Summary Statistics for Continuous Data').format(precision=2).background_gradient(cmap='Blues')
# display(numeric_summary)

In [ ]:
# DATA PREPPROCESSING (Part 2: TEXT)

# check the object columns
# Select columns of type 'object'
object_columns = df.select_dtypes(include=['object']) # 5 columns left
# print(object_columns)
# Artist, Track, Release Date, Album Name, Lyrics

# Convert object column to datetime
df['release_date'] = pd.to_datetime(df['release_date'])
print(df['release_date']) # success

In [ ]:
# check null and duplicate values

# identify null data
print("Number of null values in the dataset:\n{0}".format(df.isnull().sum())) 
# There are 26 missing value in lyrics--> will be handled

# identify duplciate data
print("\nNumber of duplicate data in the dataset:\n{0}".format(df.duplicated().sum())) # 23 duplicate values

In [ ]:
# impute the 52 songs

# DATA has 1000,20: 20=15(int/float)+4(object/string)+1(datetime)

import lyricsgenius
genius=lyricsgenius.Genius("jCaG2w1CCDLgiXm8JGamJxOdA1Z8eMEKYJhjecctVLM270hsAJkwDBReNzhbZ4Fg", retries=3)

# try to 26 lyrics
# find the songs with missing values
missing_index = df[df.isnull().any(axis=1)].index

# print("Indexes of rows with missing values:")
missing_index_list=list(missing_index)
# print(missing_index_list) # [75, 80, 147, 168, 214, 254, 395, 399, 448, 506, 516, 539, 615, 645, 648, 649, 664, 714, 761, 778, 805, 808, 811, 815, 817, 829]

# impute it manually with the website
for i in missing_index_list:
    try:
        # Attempt to search for the song
        song = genius.search_song(df.loc[i, 'track_name'].split("(", 1)[0], df.loc[i, 'artist'])
        if song:
            # IMPUTE IT 
            df.loc[i, 'lyrics']=song.lyrics
            # temp+=1
        else:
            song2 = genius.search_song(df.loc[i, 'track_name'].split()[0], df.loc[i, 'artist'])
            if song2:
                # IMPUTE IT 
                df.loc[i, 'lyrics']=song2.lyrics
                # temp+=1

    except Exception as e:
        print("Error occurred:", e)


In [ ]:
print("Number of null values in the dataset:\n{0}".format(df.isnull().sum())) 

In [ ]:
# impute the reamining songs:

# missing_index = df[df.isnull().any(axis=1)].index

# print("Indexes of rows with missing values:")
missing_index_list=[210, 1471, 1668, 2191]
# print(missing_index_list)

# 210 is don omar as artist
# the rest are just title
# 1668 snowfall
# 2191 river flows in you
# 1471 is intro
# df.loc[210, 'track_name']


# manual imputation by changing artist name
# impute it manually with the website
for i in missing_index_list:
    if i==210:
        artist="Don Omar"
        # song = genius.search_song('danza kuduro', "Don Omar")
        # if song:
        #     # IMPUTE IT 
        #     df.loc[i, 'Lyrics']=song.lyrics
    elif i==1471: # artist name
        artist="The xx"
    elif i==1668:
        artist="Øneheart & reidenshi"
    elif i==2191:
        artist="Yiruma"
    try:
        # Attempt to search for the song
        song = genius.search_song(df.loc[i, 'track_name'].split()[0], artist=artist)
        if song:
            # IMPUTE IT 
            df.loc[i, 'lyrics']=song.lyrics

    except Exception as e:
        print("Error occurred:", e)

print(df[df.isnull().any(axis=1)].index)

# remaining ones are instrumental + korean

In [ ]:
# remove duplicate values
# print("\nNumber of duplicate data in the dataset:\n{0}".format(df.duplicated().sum())) # No duplicate values

df[df.duplicated()]
# habits, don't blame me(858 or 1117), (1560 or 1561)

# drop 858 and 1560

df.drop([858, 1560],inplace=True)

In [ ]:
print("\nNumber of duplicate data in the dataset:\n{0}".format(df.duplicated().sum())) # 23 duplicate values

### EDA

* provide descriptive statistics 

* univariate

* outlier detection through boxplots for continuous data
* frequency analysis through barplot for discrete data
* outlier and unknown removal

* multivariate: pairwise scaterplot(predictor vs y) to see trends, correlation analysis to check multicolinarity (heatmaps)

* Standarizations and multicolinearity?

* write conclusions

In [ ]:
# Descriptive/ Summary statistics for continuous data (15 columns)
numeric_summary=df.describe().style.set_caption('Summary Statistics for Continuous Data').format(precision=2).background_gradient(cmap='Blues')
display(numeric_summary)

In [ ]:
df.select_dtypes(include=['object'])

In [ ]:
# Frequency analysis
import seaborn as sns

# frequency analysis for categorical variable and encoded numerical variables:
import matplotlib.pyplot as plt

# artist is uniformly distributed
# track is an issue now
# album is interesing though

test=df['album'].value_counts().head(10)

test_df=test.reset_index()
test_df.columns=['Category', 'Frequency']

# Plot the frequency distribution using a boxplot
plt.figure(figsize=(8, 6))
sns.barplot(y='Frequency', x='Category' ,data=test_df)
plt.xlabel('Album')
plt.title('Frequency Distribution of Album')
plt.xticks(rotation=30)

plt.tight_layout()
plt.show()



In [ ]:
test=df['track_name'].value_counts().head(10)

test_df=test.reset_index()
test_df.columns=['Category', 'Frequency']

# Plot the frequency distribution using a boxplot
plt.figure(figsize=(8, 6))
sns.barplot(y='Frequency', x='Category' ,data=test_df)
plt.xlabel('Track')
plt.title('Frequency Distribution of Track')
plt.xticks(rotation=30)

plt.tight_layout()
plt.show()

In [ ]:
test=df['artist'].value_counts().head(10)

test_df=test.reset_index()
test_df.columns=['Category', 'Frequency']

# Plot the frequency distribution using a boxplot
plt.figure(figsize=(8, 6))
sns.barplot(y='Frequency', x='Category' ,data=test_df)
plt.xlabel('Artist')
plt.title('Frequency Distribution of Artist')
plt.xticks(rotation=30)

plt.tight_layout()
plt.show()

In [ ]:
# univariate EDA
# numerical analysis
import matplotlib.pyplot as plt
import seaborn as sns

num_rows = 5
num_cols = 3
fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 12))

# print(df.select_dtypes(include=['number']).columns)

# exclude poutcome and y
# Plot each numeric column's box plot
for i, column in enumerate(df.select_dtypes(include=['number']).columns):
    row = i // num_cols
    col = i % num_cols
    if row < num_rows and col < num_cols:
        sns.boxplot(x=df[column], ax=axes[row, col])
        axes[row, col].set_title(column)

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
# multivariate EDA (takes 3 min)

# Multivariate analysis- print pairwise scatterplot for each x to y
# Warning: takes a long time to run

# import seaborn as sns
sns.pairplot(df)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming X is your DataFrame containing numeric features

# Select only numeric features
numeric_features = df.select_dtypes(include=['float64', 'int64'])

# Create pairplot
sns.pairplot(numeric_features)
plt.title('Pairplot of Numeric Features')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming X is your DataFrame containing numeric features

# Select only numeric features
numeric_features = df.select_dtypes(include=['float64', 'int64'])

# Calculate the correlation matrix
correlation_matrix = numeric_features.corr()

# Display the correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation Heatmap of Numeric Features')
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm
import re
from langdetect import detect
from contractions import fix
from nltk.stem import WordNetLemmatizer


# Download NLTK resources if not already installed
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Define a function to check if the text is in English
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False  # Return False if language detection fails

# Initialize BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def clean_lyrics(lyrics):
    # Check if the lyrics are a string
    if isinstance(lyrics, str):
        if len(lyrics.split('\n', 1)) > 1:
            # If there are at least two elements after split, get the second element and omit the last 8 characters
            cleaned_lyrics = lyrics.split('\n', 1)[1][:-8]
            # Remove square brackets and the content inside them
            cleaned_lyrics = re.sub(r'\[.*?\]', '', cleaned_lyrics)
            # Remove round brackets but keep their content intact, replace "\n" with " ", and lowercase every word
            cleaned_lyrics = re.sub(r'\(|\)', '', cleaned_lyrics).replace("\n", " ").lower()

            # Remove punctuation
            cleaned_lyrics = re.sub(r'[^\w\s#]', '', cleaned_lyrics)

            # Tokenization using BertTokenizer
            tokens = tokenizer.tokenize(cleaned_lyrics)

            # Join tokens back into a string
            cleaned_lyrics = ' '.join(tokens)

            # strip the spaces
            cleaned_lyrics = cleaned_lyrics.strip()

            # Check if the cleaned lyrics are in English
            return cleaned_lyrics if is_english(cleaned_lyrics) else ''
        else:
            return ''  # Return None if split_lyrics has fewer than 2 elements
    else:
        return ''  # Return None if lyrics is not a string

# Load and preprocess data
# df = pd.read_csv("your_dataset.csv")
df['lyrics'] = df['lyrics'].apply(clean_lyrics)

In [ ]:
df['lyrics']

In [ ]:
# export it
df.to_csv('processed_data.csv', index=False)